In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer, InputExample, losses, models
from torch.utils.data import DataLoader
from huggingface_hub import login

In [ ]:
%cd /content/drive/MyDrive/

/content/drive/MyDrive


In [ ]:
# === CONFIGURATION ===
HF_TOKEN = "YOURHFTOKEN"
DATASET_NAME = "Alexhuou/embedder_train_doc"
OUTPUT_DIR = "MNLP_output"
PUSH_NAME = "Alexhuou/embedder_model_STxmmluV3"

In [ ]:
# === Connexion Hugging Face
login(token=HF_TOKEN)

In [ ]:
!pip install -U datasets huggingface_hub fsspec

  Using cached fsspec-2025.5.1-py3-none-any.whl.metadata (11 kB)


In [ ]:
from datasets import load_dataset
import random
import pandas as pd

# Load the test split of MMLU (only using for retrieval, no knowing whether right answer just subject)
mmlu = load_dataset("cais/mmlu", "all", split="test")

# Convert to DataFrame for easier manipulation
df = pd.DataFrame(mmlu)

# Group by subject
subject_groups = df.groupby("subject")

# Create positive and negative pairs
data = []
for subject, group in subject_groups:
    questions = group["question"].tolist()
    if len(questions) < 2:
        continue
    for i in range(min(len(questions), 100)):  # limit samples per subject
        q = questions[i]
        # Pick a different sample in the same subject
        q_positive = random.choice([x for x in questions if x != q])

        # Pick a sample from a different subject
        other_subjects = [s for s in subject_groups.groups.keys() if s != subject]
        random_other = random.choice(other_subjects)
        q_negative = random.choice(subject_groups.get_group(random_other)["question"].tolist())

        data.append({
            "anchor": q,
            "positive": q_positive,
            "negative": q_negative,
            "subject": subject
        })

# Convert to DataFrame
triplet_df = pd.DataFrame(data)
triplet_df.head()


README.md:   0%|          | 0.00/53.2k [00:00<?, ?B/s]

dataset_infos.json:   0%|          | 0.00/138k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/3.50M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/408k [00:00<?, ?B/s]

dev-00000-of-00001.parquet:   0%|          | 0.00/76.5k [00:00<?, ?B/s]

...

Generating test split:   0%|          | 0/14042 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1531 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/285 [00:00<?, ? examples/s]

Generating auxiliary_train split:   0%|          | 0/99842 [00:00<?, ? examples/s]

,anchor,positive,negative,subject
0,Find the degree for the given field extension ...,The set of all nth roots of unity under multip...,Cultural restructuring has involved:,abstract_algebra
1,"Let p = (1, 2, 5, 4)(2, 3) in S_5 . Find the i...",Statement 1 | The unity of a subring must be t...,Romulo is on a crowded bus and he sees a woman...,abstract_algebra
2,Find all zeros in the indicated finite field o...,The set of integers Z with the binary operatio...,A 65-year-old female is admitted to the hospit...,abstract_algebra
3,Statement 1 | A factor group of a non-Abelian ...,Find the degree for the given field extension ...,This question refers to the following informat...,abstract_algebra
4,Find the product of the given polynomials in t...,Find the degree for the given field extension ...,This question refers to the following informat...,abstract_algebra


In [ ]:
triplet_df.describe()

,anchor,positive,negative,subject
count,5700,5700,5700,5700
unique,5645,4308,4270,57
top,Which of the following statements is correct?,"The suspension cable of a 1,000 kg elevator sn...",Which of the following statements is false?,abstract_algebra
freq,4,7,7,100


In [ ]:
from datasets import Dataset

hf_dataset = Dataset.from_pandas(triplet_df)

# Define your repo name (replace with your actual HF username and desired repo name)
repo_name = DATASET_NAME

# Push to the Hub
hf_dataset.push_to_hub(repo_name)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/6 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/386 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Alexhuou/embedder_train_doc/commit/f41e8ac7f4593ff8c3b4a524d1779ebe0092f596', commit_message='Upload dataset', commit_description='', oid='f41e8ac7f4593ff8c3b4a524d1779ebe0092f596', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/Alexhuou/embedder_train_doc', endpoint='https://huggingface.co', repo_type='dataset', repo_id='Alexhuou/embedder_train_doc'), pr_revision=None, pr_num=None)

In [ ]:
# === Charger le dataset
dataset = load_dataset(DATASET_NAME)
train_data = dataset["train"]

README.md:   0%|          | 0.00/386 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.98M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5700 [00:00<?, ? examples/s]

In [ ]:
# === Transformer en liste d'exemples pour TripletLoss
train_examples = [
    InputExample(texts=[ex["anchor"], ex["positive"], ex["negative"]])
    for ex in train_data
]

In [ ]:
len(train_examples)

5700

In [ ]:
# === Charger le modèle SentenceTransformer
model_name = "thenlper/gte-small"

word_embedding_model = models.Transformer(model_name)

pooling_model = models.Pooling(
    word_embedding_model.get_word_embedding_dimension(),
    pooling_mode_mean_tokens=True
)

# Step 3: Combine into a SentenceTransformer model
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/66.7M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/394 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [ ]:
# === Préparer le DataLoader et la Loss
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=8)
train_loss = losses.TripletLoss(model=model)

In [ ]:
len(train_dataloader)

713

In [ ]:
# === Entraînement
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=5,
    warmup_steps=10,
    output_path=OUTPUT_DIR
)

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: alex-huou (epfl-comm-proj) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
500,1.938200
1000,1.088200
1500,0.847800
2000,0.596100
2500,0.517900
3000,0.377400
3500,0.364600


In [ ]:

print(f"✅ Modèle entraîné sauvegardé dans : {OUTPUT_DIR}")

# === Push vers Hugging Face Hub
model.push_to_hub(PUSH_NAME)
print(f"📤 Modèle pushé sur Hugging Face : {PUSH_NAME}")

✅ Modèle entraîné sauvegardé dans : MNLP_output


model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

📤 Modèle pushé sur Hugging Face : Alexhuou/embedder_model_STxmmluV3
